#Distance Estimator
To estimate the real distance(unit: meter) of the object

__Input__: Bounding box coordinates(xmin, ymin, xmax, ymax)   
__Output__: 3D location z of carmera coordinates(z_loc)

## Load Module

In [1]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import time
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 
from sklearn.preprocessing import StandardScaler
from custom_datasets import CustomDataset
from sklearn.metrics import mean_squared_error

In [2]:
os.makedirs('./weights', exist_ok=True)

## Dataset

In [3]:
df_train = pd.read_csv('../datasets/kitti_train.csv')
df_valid = pd.read_csv('../datasets/kitti_valid.csv')
df_test = pd.read_csv('../datasets/kitti_test.csv')

In [4]:
df_train['class'].unique()

array(['person', 'car', 'truck', 'train', 'bicycle', 'Misc'], dtype=object)

In [5]:
# onehot encoding
class_dummy = pd.get_dummies(df_train['class'])
df_train = pd.concat([df_train, class_dummy], axis=1)

class_dummy = pd.get_dummies(df_valid['class'])
df_valid = pd.concat([df_valid, class_dummy], axis=1)

class_dummy = pd.get_dummies(df_test['class'])
df_test = pd.concat([df_test, class_dummy], axis=1)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22022 entries, 0 to 22021
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    22022 non-null  object 
 1   class       22022 non-null  object 
 2   xmin        22022 non-null  float64
 3   ymin        22022 non-null  float64
 4   xmax        22022 non-null  float64
 5   ymax        22022 non-null  float64
 6   angle       22022 non-null  float64
 7   zloc        22022 non-null  float64
 8   weather     22022 non-null  object 
 9   depth_y     22022 non-null  int64  
 10  depth_mean  22022 non-null  float64
 11  depth_x     22022 non-null  int64  
 12  depth_min   22022 non-null  float64
 13  width       22022 non-null  float64
 14  height      22022 non-null  float64
 15  Misc        22022 non-null  uint8  
 16  bicycle     22022 non-null  uint8  
 17  car         22022 non-null  uint8  
 18  person      22022 non-null  uint8  
 19  train       22022 non-nul

In [7]:
variable = ['width', 'height', 'depth_mean', 'depth_min', 'Misc', 'bicycle', 'car', 'person', 'train', 'truck']
batch_sz = 8
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# train
train_dataset = CustomDataset(df_train, variable)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_sz, shuffle=True)

# valid
valid_dataset = CustomDataset(df_valid, variable)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_sz, shuffle=True)

# train
test_dataset = CustomDataset(df_test, variable)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_sz, shuffle=False)

In [8]:
# look dataset
for idx, batch in enumerate(train_dataloader):
    if idx == 1:
        break
    print(batch[0])
    print(batch[0].shape)
    print(batch[1])

tensor([[ 24.6759,  14.4393,   8.8445,   8.5643,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [ 49.9677,  14.7209,   4.6949,   3.4180,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [ 76.0463,  64.4582,   4.1528,   3.1694,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   1.0000],
        [ 87.4794,  40.5914,   3.1512,   2.7837,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [ 66.0756,  19.0608,   4.7666,   3.0124,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [ 33.6621,  26.3322,   4.7512,   3.6130,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [130.6615,  80.1897,   2.4384,   1.7792,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000],
        [ 61.3220,  36.6520,   4.7757,   2.3093,   0.0000,   0.0000,   1.0000,
           0.0000,   0.0000,   0.0000]])
torch.Size([8, 10])
tensor([[72.6500],
        

In [9]:
# standardized data
#scalar = StandardScaler()
#X_train = scalar.fit_transform(X_train)
#y_train = scalar.fit_transform(y_train)

## Modeling

In [10]:
class DistanceEstimator(nn.Module):
    def __init__(self):
        super(DistanceEstimator, self).__init__()
        
        #Layer
        self.model = nn.Sequential(
            nn.Linear(10,32),
            nn.ReLU(),
            
            nn.Linear(32,64),
            nn.ReLU(),
            
            nn.Linear(64,128),
            nn.ReLU(),
            
            nn.Linear(128,256),
            nn.ReLU(),
            
            nn.Linear(256,128),
            nn.ReLU(),
            
            nn.Linear(128,64),
            nn.ReLU(),
            
            nn.Linear(64,32),
            nn.ReLU(),
            
            nn.Linear(32,16),
            nn.ReLU(),
            
            nn.Linear(16,1)
        )

    def forward(self, x):
        out = self.model(x)
        return out

## Make  variable

In [11]:
model = DistanceEstimator()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       patience = 8,
                                                       mode='min', # 우리는 낮아지는 값을 기대
                                                       verbose=True)

model.to(device)

DistanceEstimator(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=128, bias=True)
    (9): ReLU()
    (10): Linear(in_features=128, out_features=64, bias=True)
    (11): ReLU()
    (12): Linear(in_features=64, out_features=32, bias=True)
    (13): ReLU()
    (14): Linear(in_features=32, out_features=16, bias=True)
    (15): ReLU()
    (16): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [12]:
# train parameters
def count_parameter(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameter(model) # 87585

87585

## Make Train, Valid function

In [13]:
def train(model, train_dataloader, idx_interval):
    model.train()
    
    train_loss = 0
    train_rmse = 0
    
    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        prediction = model(batch[0].to(device))
        loss = loss_fn(prediction, batch[1].to(device)).cpu()
        
        # Backpropagation
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
        train_rmse += np.sqrt(loss.item())
        
        if idx % idx_interval == 0:
            print("Train Epoch: {} [{}/{}] \t Train Loss(MSE): {:.4f} \t Train RMSE: {:.4f}".format(epoch, batch_sz*(idx+1), \
                                                                            len(train_dataloader)*batch_sz, \
                                                                            loss.item(), np.sqrt(loss.item())))
    
    train_loss /= len(train_dataloader)
    train_rmse /= len(train_dataloader)
        
    return train_loss, train_rmse

In [14]:
def evaluate(model, valid_dataloader):
    model.eval()
    
    valid_loss = 0
    valid_rmse = 0
    
    with torch.no_grad():
        for idx, batch in enumerate(valid_dataloader):
            predictions = model(batch[0].to(device))
            loss = loss_fn(predictions, batch[1].to(device)).cpu()
            valid_loss += loss.item()
            valid_rmse += np.sqrt(loss.item())
            
    valid_loss /= len(valid_dataloader)
    valid_rmse /= len(valid_dataloader)
    
    return valid_loss,valid_rmse

In [15]:
# Function to save the model 
def saveModel(model): 
    path = "./weights/ODD_basic.pth" 
    torch.save(model.state_dict(), path) 

## Train and Validation

In [16]:
Epoch = 100
best_rmse = 99999

train_mse_list = []
train_rmse_list = []
valid_mse_list = []
valid_rmse_list = []

for epoch in range(1,(Epoch+1)):
    train_mse, train_rmse = train(model, train_dataloader, 250)
    valid_mse, valid_rmse = evaluate(model, valid_dataloader)

    print("[Epoch: {} \t Valid MSE: {:.4f} \t Valid RMSE: {:.4f}]".format(epoch, valid_mse, valid_rmse))
    
    scheduler.step(valid_mse)

    # Save model
    if valid_rmse < best_rmse:
        path = "./weights/ODD_basic.pth" 
        torch.save(model.state_dict(), path) # 모델의 가중치만 저장 구조는 저장 x..?
        
    train_mse_list.append(train_mse)
    train_rmse_list.append(train_rmse)
    valid_mse_list.append(valid_mse)
    valid_rmse_list.append(valid_rmse)

Train Epoch: 1 [8/22024] 	 Train Loss(MSE): 1887.4092 	 Train RMSE: 43.4443
Train Epoch: 1 [2008/22024] 	 Train Loss(MSE): 125.1494 	 Train RMSE: 11.1870
Train Epoch: 1 [4008/22024] 	 Train Loss(MSE): 17.0961 	 Train RMSE: 4.1347
Train Epoch: 1 [6008/22024] 	 Train Loss(MSE): 81.1805 	 Train RMSE: 9.0100
Train Epoch: 1 [8008/22024] 	 Train Loss(MSE): 34.0513 	 Train RMSE: 5.8353
Train Epoch: 1 [10008/22024] 	 Train Loss(MSE): 5.0420 	 Train RMSE: 2.2454
Train Epoch: 1 [12008/22024] 	 Train Loss(MSE): 83.3136 	 Train RMSE: 9.1276
Train Epoch: 1 [14008/22024] 	 Train Loss(MSE): 25.9815 	 Train RMSE: 5.0972
Train Epoch: 1 [16008/22024] 	 Train Loss(MSE): 12.1833 	 Train RMSE: 3.4905
Train Epoch: 1 [18008/22024] 	 Train Loss(MSE): 18.1848 	 Train RMSE: 4.2644
Train Epoch: 1 [20008/22024] 	 Train Loss(MSE): 6.0508 	 Train RMSE: 2.4598


KeyboardInterrupt: 

# Epoch visualization

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1,1,1)
ax1.plot(train_mse_list, ls='-', color='blue', label='train')
ax2 = ax1.twinx()
ax2.plot(valid_mse_list, ls='--', color='red', label='valid')
ax1.set_title('MSE error')
ax1.legend(loc='upper right')
ax2.legend(loc='upper left')
plt.show()


In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1,1,1)
ax1.plot(train_rmse_list, ls='-', color='blue', label='train')
ax2 = ax1.twinx()
ax2.plot(valid_rmse_list, ls='--', color='red', label='valid')
ax1.set_title('RMSE error')
ax1.legend(loc='upper right')
ax2.legend(loc='upper left')
plt.show()

# Predict Test

In [17]:
# 가중치 가져오기
model = DistanceEstimator()
model.load_state_dict(torch.load('./weights/ODD_basic.pth'))
model.eval()
model.to(device)

DistanceEstimator(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=128, bias=True)
    (9): ReLU()
    (10): Linear(in_features=128, out_features=64, bias=True)
    (11): ReLU()
    (12): Linear(in_features=64, out_features=32, bias=True)
    (13): ReLU()
    (14): Linear(in_features=32, out_features=16, bias=True)
    (15): ReLU()
    (16): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [18]:
test_mse, test_rmse = evaluate(model, test_dataloader)
print('Test MSE: {:4f} \t Test RMSE: {:4f}'.format(test_mse, test_rmse))

Test MSE: 17.149220 	 Test RMSE: 3.442605


In [19]:
predict_zloc = model(torch.FloatTensor(df_test[variable].values).to(device))

In [20]:
df_test['predict'] = predict_zloc.cpu().detach().numpy()
df_test[['zloc','predict']].head(50)

,zloc,predict
0,44.91,45.816387
1,33.82,36.278297
2,3.98,6.413482
3,11.39,13.523090
4,13.04,13.328783
5,21.00,20.019951
6,21.09,19.818882
7,48.50,41.965740
8,22.45,24.500395
9,51.44,53.706333


from sklearn.metrics import mean_squared_error
def train_model(model, train_dataloader, valid_dataloader, loss_fn, lr=1e-5, batch_size=512, epochs=100, validate=False):

  
  # Convert model parameters and buffers to CPU or Cuda
  model.to(device)

  best_rmse = np.Inf
  print("Begin training...") 
  for epoch in range(1, epochs+1): 
    running_train_loss = 0.0 
    running_rmse = 0.0 
    running_vall_loss = 0.0 
    total = 0 

    for batch_ind, samples in enumerate(train_dataloader):
      x_train, y_train = samples
      optimizer.zero_grad()
      pred = model.forward(x_train)
      train_loss = loss_fn(pred, y_train)
      train_loss.backward()
      optimizer.step()
      running_train_loss += train_loss.item()

    train_loss_value = running_train_loss/len(train_dataloader)
    with torch.no_grad(): 
      model.eval() 
      for data in valid_dataloader: 
        inputs, outputs = data 
        predicted_outputs = model(inputs) 
        val_loss = loss_fn(predicted_outputs, outputs) 
      
        # The label with the highest value will be our prediction 
        running_vall_loss += val_loss.item()  
        total += outputs.size(0) 
        rmse = mean_squared_error(outputs, predicted_outputs)**0.5
        running_rmse += rmse

    # Calculate validation loss value 
    val_loss_value = running_vall_loss/len(valid_dataloader)  
    rmse = running_rmse / total

    if rmse < best_rmse:
      saveModel(model)
      best_rmse = rmse

    # Print the statistics of the epoch 
    print('Epoch {0}/{1} - loss: {2:.4f} / val_loss: {3:.4f} - RMSE: {4:.4f}'.format(epoch, epochs, train_loss_value, val_loss_value,rmse))

train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)
train_dataloader = DataLoader(train_dataset, batch_size=2)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)

model = DistanceEstimator()
#optimizer = torch.optim.Adam(model.parameters, lr=1e-5)
loss_func = nn.MSELoss()

train_model(model, train_dataloader, valid_dataloader, loss_func, epochs=100, batch_size=2048)

##Predict

def predict(test_dataloader): 
    # Load the model that we saved at the end of the training loop 
    model = DistanceEstimator()
    path = "NetModel.pth" 
    model.load_state_dict(torch.load(path)) 
     
    running_rmse = 0 
    total = 0 
    pred = []
 
    with torch.no_grad(): 
      for data in test_dataloader: 
        inputs, outputs = data 
        outputs = outputs.to(torch.float32) 
        predicted_outputs = model(inputs) 
        pred.append(float(predicted_outputs))
        total += outputs.size(0) 
        rmse = mean_squared_error(outputs, predicted_outputs)**0.5
        running_rmse += rmse
 
      print('RMSE:',running_rmse / total)
    return pred

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=2)

pred = predict(test_dataset)

#Result with prediction
df_test['zloc_pred'] = pred
df_test